In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from sklearn.model_selection import train_test_split

K.set_image_dim_ordering('th')

In [108]:
# load (downloaded if needed) the MNIST dataset
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
train.shape

(42000, 785)

In [109]:
train_x = train.iloc[:,1:].values.astype('float32') # all pixel values
train_y = train.iloc[:,0].values.astype('int32') # only labels i.e targets digits
test = test.values.astype('float32')

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [110]:
# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')
test=test.reshape(test.shape[0], 1, 28, 28).astype('float32')

# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
y_train.shape

(33600,)

In [112]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

num_classes =y_test.shape[1]
X_train.shape

(33600, 1, 28, 28)

In [96]:
def larger_model():
    # create model
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(1, 28, 28), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [35]:
model = larger_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_train, y_train), epochs=10, batch_size=200)
# Final evaluation of the model
scores = model.evaluate(X_train, y_train, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Train on 42000 samples, validate on 42000 samples
Epoch 1/10
42000/42000 [==============================] - 84s 2ms/step - loss: 0.3018 - acc: 0.9148 - val_loss: 0.1093 - val_acc: 0.9687
Epoch 2/10
42000/42000 [==============================] - 84s 2ms/step - loss: 0.0894 - acc: 0.9732 - val_loss: 0.0606 - val_acc: 0.9824
Epoch 3/10
42000/42000 [==============================] - 78s 2ms/step - loss: 0.0613 - acc: 0.9813 - val_loss: 0.0376 - val_acc: 0.9892
Epoch 4/10
42000/42000 [==============================] - 80s 2ms/step - loss: 0.0480 - acc: 0.9856 - val_loss: 0.0349 - val_acc: 0.9903
Epoch 5/10
42000/42000 [==============================] - 79s 2ms/step - loss: 0.0397 - acc: 0.9879 - val_loss: 0.0308 - val_acc: 0.9904
Epoch 6/10
42000/42000 [==============================] - 78s 2ms/step - loss: 0.0330 - acc: 0.9895 - val_loss: 0.0190 - val_acc: 0.9948
Epoch 7/10
42000/42000 [==============================] - 73s 2ms/step - loss: 0.0271 - acc: 0.9914 - val_loss: 0.0148 - val_acc

In [41]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [59]:
predictions = model.predict_classes(test, verbose=0)

submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
                         "Label": predictions})
submissions.to_csv("test_predictions.csv", index=False, header=True)

In [58]:
test.shape

(28000, 1, 28, 28)